## Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy


warnings.filterwarnings('ignore')

## Reading images

In [ ]:
all_images = []

for image in os.listdir("data"):
    image_path = os.path.join("data", image)
    all_images.append(image_path)

## Showing sample images

In [ ]:
selected_images = random.sample(all_images, 6)

fig, ax = plt.subplots(1, 6, figsize=(35, 35))

for i, image_path in enumerate(selected_images):
    img = mpimg.imread(image_path)
    ax[i].imshow(img)
    ax[i].axis('off')

plt.show()

## Loading training images

In [ ]:
train_images = [img_to_array(load_img(path)) for path in all_images]
train_images = np.array(train_images, dtype='float32')

train_images = (train_images - 127.5) / 127.5
train_images = train_images.reshape(train_images.shape[0], 64, 64, 3)

## GAN generator

In [ ]:
generator = Sequential(name='generator')

generator.add(layers.Dense(8 * 8 * 512, input_dim=100))
generator.add(layers.ReLU())
generator.add(layers.Reshape((8, 8, 512)))

generator.add(layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.02)))
generator.add(layers.ReLU())

generator.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.02)))
generator.add(layers.ReLU())

generator.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.02)))
generator.add(layers.ReLU())

generator.add(layers.Conv2D(3, (4, 4), padding='same', activation='tanh'))

generator.summary()

## GAN discriminator

In [ ]:
discriminator = Sequential(name='discriminator')
input_shape = (64, 64, 3)

discriminator.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape))
discriminator.add(layers.BatchNormalization())
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(layers.BatchNormalization())
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
discriminator.add(layers.BatchNormalization())
discriminator.add(layers.LeakyReLU(alpha=0.2))

discriminator.add(layers.Flatten())
discriminator.add(layers.Dropout(0.3))

discriminator.add(layers.Dense(1, activation='sigmoid'))

discriminator.summary()

## DCGAN class implementation

In [ ]:
class DCGAN(keras.Model):
    def __init__(self, generator, discriminator, latent_dim):
        super(DCGAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.discriminator_loss_metric = keras.metrics.Mean(name='discriminator_loss')
        self.generator_loss_metric = keras.metrics.Mean(name='generator_loss')
        
    @property
    def metrics(self):
        return [self.discriminator_loss_metric, self.generator_loss_metric]
    
    def compile(self, g_opt, d_opt, loss_fn):
        super(DCGAN, self).compile()
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.loss_fn = loss_fn
        
    def train_step(self, real_imgs):
        batch_size = tf.shape(real_imgs)[0]
        noise = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        with tf.GradientTape() as tape:
            pred_real = self.discriminator(real_imgs, training=True)
            real_labels = tf.ones((batch_size, 1)) + 0.05 * tf.random.uniform((batch_size, 1))
            discriminator_loss_real = self.loss_fn(real_labels, pred_real)
            
            fake_imgs = self.generator(noise, training=True)
            pred_fake = self.discriminator(fake_imgs, training=True)
            fake_labels = tf.zeros((batch_size, 1))
            discriminator_loss_fake = self.loss_fn(fake_labels, pred_fake)
            
            discriminator_loss = (discriminator_loss_real + discriminator_loss_fake) / 2
            
        gradients = tape.gradient(discriminator_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(gradients, self.discriminator.trainable_variables))
        
      
        labels = tf.ones((batch_size, 1))
        with tf.GradientTape() as tape:
            fake_imgs = self.generator(noise, training=True)
            pred_fake = self.discriminator(fake_imgs, training=True)
            generator_loss = self.loss_fn(labels, pred_fake)
            
        gradients = tape.gradient(generator_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(gradients, self.generator.trainable_variables))
        
        self.discriminator_loss_metric.update_state(discriminator_loss)
        self.generator_loss_metric.update_state(generator_loss)
        
        return {'discriminator_loss': self.discriminator_loss_metric.result(),
                'generator_loss': self.generator_loss_metric.result()}

## DCGANMonitor class implementation

In [ ]:
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs=25, latent_dim=100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        self.noise = tf.random.normal([25, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        gen_imgs = self.model.generator(self.noise)
        gen_imgs = (gen_imgs * 127.5) + 127.5

        fig = plt.figure(figsize=(8, 8))
        for i in range(self.num_imgs):
            plt.subplot(5, 5, i+1)
            img = array_to_img(gen_imgs[i])
            plt.imshow(img)
            plt.axis('off')
        plt.show()
    
    def on_train_end(self, logs=None):
        self.model.generator.save('generator.h5')
        self.model.discriminator.save('discriminator.h5')

dcgan = DCGAN(generator, discriminator, 100)

## Compiling and fitting the model

In [ ]:
dcgan.compile(
    g_opt=Adam(learning_rate=0.0003, beta_1=0.5),
    d_opt=Adam(learning_rate=0.0001, beta_1=0.5),
    loss_fn=BinaryCrossentropy()
)

N_EPOCHS = 55
dcgan.fit(train_images, epochs=N_EPOCHS, callbacks=[DCGANMonitor()])



## Making new anime faces

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import array_to_img
import matplotlib.pyplot as plt

generator = tf.keras.models.load_model('generator.h5')
discriminator = tf.keras.models.load_model('discriminator.h5')

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
generator_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

discriminator_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

generator.compile(optimizer=generator_optimizer, loss=generator_loss)

discriminator.compile(optimizer=discriminator_optimizer, loss=discriminator_loss)

def create_dcgan(generator, discriminator):
    discriminator.trainable = False 
    dcgan_input = tf.keras.Input(shape=(100,)) 
    generated_image = generator(dcgan_input)
    dcgan_output = discriminator(generated_image)
    dcgan = tf.keras.Model(dcgan_input, dcgan_output)
    dcgan.compile(optimizer=generator_optimizer, loss=generator_loss)
    return dcgan

dcgan = create_dcgan(generator, discriminator)

noise = tf.random.normal([1, 100]) 

g_img = generator.predict(noise)

g_img = (g_img * 127.5) + 127.5

img = array_to_img(g_img[0])

fig = plt.figure(figsize=(3, 3))
plt.imshow(img)
plt.axis('off')
plt.show()